In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 
import dtwsom
from minisom import MiniSom
import pickle

/home/kenan/.pyenv/versions/3.9.2/lib/python3.9/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [3]:
with open('daily_NO2.pkl', 'rb') as f:
    data_NO2_df = pickle.load(f)
dataNO2 =np.stack(data_NO2_df["NO2"])

In [4]:
with open('daily_O3.pkl', 'rb') as f:
    data_O3_df = pickle.load(f)
dataO3 =np.stack(data_O3_df["O3"])

In [5]:
dataNO2plusO3=np.array([dataO3[:1000, :], dataNO2[:1000, :]])

In [6]:
somNO2plusO3 = dtwsom.MultiDtwSom(10, 10, dataNO2.shape[1], bands =2, w = [0.5, 0.5], sigma=1, learning_rate=0.5, random_seed=10,gl_const="sakoe_chiba", scr=60)
somNO2plusO3.pca_weights_init(dataNO2plusO3)
somNO2plusO3.train_batch(dataNO2plusO3, 200, verbose=True)
weightsNO2plusO3 = somNO2plusO3.get_weights()

 [ 200 / 200 ] 100% - 9.21 it/s - 0:00:00 left  - quantization error: 78.08714906921958
